Для теста используется датасет MTS ML Cup – 322млн.899435 строк – история активности пользователей в интернете

[Скачать архив с датасетом](https://storage.yandexcloud.net/ds-ods/files/materials/124f46f0/competition_data_final_pqt.zip)



In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pandas as pd
import dask.dataframe as dd

In [2]:
DATASET = '../mts/dataset/competition_data_final_pqt'

In [26]:
%%time
spark = SparkSession.builder \
    .master('local[*]') \
    .appName('myAppName') \
    .config('spark.driver.memory','4g') \
    .config('spark.ui.port',4041) \
    .getOrCreate()
spark.sparkContext.setLogLevel('FATAL')

df = spark.read.parquet(DATASET)

df.select(F.split(df.url_host,'\.').alias('url_parts')).show(20,False)
df.groupBy('cpe_model_os_type').count().show()

print('\nTotal rows',df.count())
del(df)
spark.stop()

+--------------------------------+
|url_parts                       |
+--------------------------------+
|[apple, com]                    |
|[apple, com]                    |
|[banners, adfox, ru]            |
|[apple, com]                    |
|[instagram, com]                |
|[yastatic, net]                 |
|[apple, com]                    |
|[googleads, g, doubleclick, net]|
|[relap, io]                     |
|[yastatic, net]                 |
|[zen, yandex, ru]               |
|[apple, com]                    |
|[avatars, mds, yandex, net]     |
|[instagram, com]                |
|[interesbook, com]              |
|[ok, ru]                        |
|[online, sberbank, ru]          |
|[online, sberbank, ru]          |
|[vk, com]                       |
|[yandex, ru]                    |
+--------------------------------+
only showing top 20 rows



+-----------------+---------+
|cpe_model_os_type|    count|
+-----------------+---------+
|              iOS|122764510|
|        Apple iOS|  2275740|
|          Android|197859185|
+-----------------+---------+


Total rows 322899435
CPU times: user 16.9 ms, sys: 8.09 ms, total: 25 ms
Wall time: 5.73 s


In [ ]:
%%time
df = dd.read_parquet(DATASET)
print(df.url_host.str.split('\.').head(20))
# print(df.groupby('cpe_model_os_type').cpe_model_os_type.count().compute())
del(df)

0                    [ad, adriver, ru]
1                         [apple, com]
2          [avatars, mds, yandex, net]
3     [googleads, g, doubleclick, net]
4     [googleads, g, doubleclick, net]
5                      [i, ytimg, com]
6                        [icloud, com]
7                       [m, avito, ru]
8                          [relap, io]
9               [sun9-5, userapi, com]
10                    [ads, adfox, ru]
11                        [apple, com]
12                        [apple, com]
13                [banners, adfox, ru]
14    [googleads, g, doubleclick, net]
15    [googleads, g, doubleclick, net]
16    [googleads, g, doubleclick, net]
17                       [icloud, com]
18                  [mail, yandex, ru]
19         [syndication, realsrv, com]
Name: url_host, dtype: object


In [ ]:
%%time
df = pd.read_parquet(DATASET,columns=['url_host','cpe_model_os_type'])
print(df.url_host.str.split('\.').head(20))
print(df.groupby('cpe_model_os_type').cpe_model_os_type.count())
del(df)